# Part 5 - Data Modeling

## Get Data

In [ ]:
# seed
import random

seed_id = 11774987
random_state = random.seed(seed_id)
random_state

In [ ]:
import shutil

# Source file path (within your Drive)
source_file = '/content/drive/MyDrive/Colab Notebooks/train_test_split.pkl'

# Destination path (root of your Drive)
destination_path = 'train_test_split.pkl'

# Copy the file
shutil.copy(source_file, destination_path)

'train_test_split.pkl'

In [ ]:
import pickle

# Load the data
with open('train_test_split.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

## Logistic Regression

In [ ]:
# model, predict, evaluate, and plot
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

model = LogisticRegression(solver='liblinear', random_state=random_state)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[119  24]
 [ 19 138]]
0.8566666666666667


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=random_state)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[120  23]
 [ 19 138]]
0.86


## Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print('accuracy:', accuracy_score(predictions, y_test))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

accuracy: 0.86
[[120  23]
 [ 19 138]]
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85       143
         1.0       0.86      0.88      0.87       157

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300



## Model Fine-Tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

hyperparameters = {'max_depth': [2, 3],
              'min_samples_split': [4, 5],
              'min_samples_leaf': [4, 5],
              'bootstrap': [True, False],
              'criterion': ['entropy', 'gini']}

grid_search = GridSearchCV(estimator = RandomForestClassifier(),
                           param_grid = hyperparameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('best accuracy', best_accuracy)
print('best parameters', best_parameters)

best accuracy 0.8671428571428571
best parameters {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 4}


## Final Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = RandomForestClassifier(random_state=random_state).set_params(**best_parameters) # * args, ** kwargs
model.fit(X_train.values, y_train)
predictions = model.predict(X_test.values)
print('accuracy:', accuracy_score(predictions, y_test))

accuracy: 0.87


## Confusion Matrix

In [ ]:
print(confusion_matrix(y_test, predictions))

[[115  28]
 [ 11 146]]


## Precision Recall

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.91      0.80      0.86       143
         1.0       0.84      0.93      0.88       157

    accuracy                           0.87       300
   macro avg       0.88      0.87      0.87       300
weighted avg       0.87      0.87      0.87       300



## Bias Variance

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
    model,
    X_train.values, # Convert X_train to NumPy array
    y_train.values, # Convert y_train to NumPy array
    X_test.values, # Convert X_test to NumPy array
    y_test.values, # Convert y_test to NumPy array
    loss='0-1_loss',
    random_seed=random_state)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)

# Average expected loss: 0.087
# Average bias: 0.080
# Average variance: 0.034

Average expected loss: 0.134
Average bias: 0.127
Average variance: 0.022


In [ ]:
X_test.head()

,watch_time,avg_view_duration,click_through_rate,interest
612,0.844716,0.776083,0.284810,0.188
660,-0.122425,0.811731,-0.196283,0.188
168,-3.265048,0.091849,-1.421401,0.097
350,3.723919,0.001996,1.944370,0.193
187,-1.894662,0.220398,-0.605270,0.244


In [ ]:
import pandas as pd

sample_to_predict = pd.Series({"watch_time": -8.1, "avg_view_duration": 1.4, "click_through_rate": -0.7, "interest": 0})
sample_to_predict = pd.DataFrame([sample_to_predict])
model.predict_proba(sample_to_predict.values)

array([[0.91841801, 0.08158199]])

In [ ]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
import shutil

# Source file path (within your Drive)
source_file = 'model.pkl'

# Destination path (root of your Drive)
destination_path = '/content/drive/MyDrive/Colab Notebooks/model.pkl'

# Copy the file
shutil.copy(source_file, destination_path)

'/content/drive/MyDrive/Colab Notebooks/model.pkl'